## PyTrix Series
# Handling Missing Data
## PyTrix #7: Various Techniques to Handle Missing Data


In the last series of PyTrix we touched on Missing data; Here's an excerpt from the last PyTrix "When we observe missing values within our data, it is as a result of no data value being stored in the particular variable for an observation. Missing data is extremely common in practice and can have a large effect on the conclusions that can be drawn from the data, hence why a huge portion of Data Scientist time is attributed towards data cleaning." 

We explored ways to detect missing Data, but in that post I did not give a clear solution as to how we should go about combating the problem of missing data. In this post I hope to run through some strategies that I've found useful and because this is PyTrix, of-course I am going to share some useful Python tricks we could use to perform these task. 

We will be using the infamous Titanic dataset from Kaggle that you can access [here](https://www.kaggle.com/c/titanic/data?select=train.csv). 


In [1]:
import numpy as np
import pandas as pd

In [2]:
# reading the data
df = pd.read_csv("../data/titanic_raw/train.csv")
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


We can see some `NaN` values in the Age and Cabin columns, but it will be difficult to see how much missing data we have by lookng at our data in this way. A better solution would be to use `df.info()` which will give us the non-null count and data type of each column - See the [Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html)  

In [3]:
# information on the columns
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


All columns have 891 observations, but we only have 714/891 non-null observations in the Age column, 204/891 in the Cabin column and 889/891 for the Embarked column. In an ideal world, we'd of done some intensive analysis on the data we currently have because we want to attempt to understand why the data has gone missing in the first place. Additionally, speaking to a domain expert (if you aren't one) is also a mandatory step when dealing with missing values so that whatever technique we use is sensible and we can still derive accurate predictions from our model.

The reason for this is simple, missing data reduces the representativeness of the sample and can therefore distort inferences about the population, hence if we are going to deal with them we want to ensure we are providing our models with an accurate representation of the population.

Since our purpose is to explore various techniques to handle missing data, we can assume the prior requirements have been carried out.

## Deletion
The easiest way to deal with missing data is to delete them. Also referred to as listwise deletion, in this method we remove an entire row from the analysis if it has a single value missing. 

In [4]:
df2 = df.copy() 

print("Before removing rows with missing values")
print(df2.shape)
print("After removing rows with missing values")
df2.dropna(inplace=True)
print(df2.shape)

Before removing rows with missing values
(891, 12)
After removing rows with missing values
(183, 12)


Removing the rows from our data can cause more harm than it does justice. In this instance, we dropped 708 rows (20.5% of observations) which is a significant amount of data considering our sample size. This is problematic since reducing the amount of data reduces statistical power of our analysis since this in part relies on a high sample size. 

Additionally, if the reason for the missing data is not random we would be creating a bias in the findings if we decide to perform listwise deletion. An example of non-random missing data is when intrusive questions (i.e. How much do you earn?) are asked in questionnaires. Many people may not feel comfortable to share that information but would have no problem answering other questions. The bias may occur because the people that do decide to answer every question including the intrusive questions may have very different characteristics than the participants who do not feel comforable to answer the intrusive ones. 

## Imputation

The next few techniques focus more on imputation which is the process of replacing missing data with substituted values. Listwise deletion, although the most common way to deal with missing data due to its simplicity, comes with its pitfalls. Imputation can be viewed as a way to avoid these pitfalls. 



### Hot-Deck 

This is where a missing value is imputed from a randomly selected similar record. A form of hot-deck imputation is Last Observation Carried Forward (LOCF imputation). What LOCF does is it finds the first missing value then imputes the missing value using the cell value that is immedietly prior. This method aligns with the belief that the best guess is that the data has not changed from the last measurement. We will implement it in Python. 

In [5]:
df_hot_deck = df.copy()
df[887:]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


Now we will perform LOCF on this dataframe and this should impute all the missing values using the cell above the missing value

In [6]:
df_hot_deck.fillna(method="ffill", inplace=True)
df_hot_deck[887:]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,19.0,1,2,W./C. 6607,23.45,B42,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,C148,Q


Using this method to impute data is known to increase the risk of increasing bias and potentially leading to false conclusions on our data, therefore it is not often recommended - I am not very sure if people use this in practice. 

### Statiscal Methods 

This may not technically be the correct term to use to describe these techniques, however I wanted an umbrella term that would capture all the different ways we could use statistics to impute our data. Common statistics are mean, median and mode.

In [7]:
# View all rows with a missing age
df3 = df.copy()
missing_age = df3[df3["Age"].isna()].index

df3[df3["Age"].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


In [8]:
# imputing all rows with missing age with the mean age
df3["Age"].fillna(df3["Age"].mean(), inplace=True)
df3.loc[missing_age]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,29.699118,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,29.699118,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,29.699118,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,29.699118,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,29.699118,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,29.699118,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,29.699118,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,29.699118,0,0,349217,7.8958,NaN,S


Mean imputation attenuates any correlation that invovle any features that are imputed - there is guranteed to be no relationship between the imputed variable and any other measured variable - The same can be said of using the mode, but it does work pretty well with categorical data. 

In [9]:
df3["Embarked"].value_counts(dropna=False)

S      644
C      168
Q       77
NaN      2
Name: Embarked, dtype: int64

In [10]:
missing_embarked = df3[df3["Embarked"].isna()].index
df3[df3["Embarked"].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [11]:
df3["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)
df3.loc[missing_embarked]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,S
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,S


### Model Imputation 

Again, this may be bad terminology that I am using to describe this phenomena. A popular way to approach missing data imputation is to use a model to predict missing values. There are many models to select from though the kNN has proven to be quite effective (often referred to as nearest neighbors imputation). 

In [14]:
from sklearn.impute import KNNImputer

In [29]:
X = df[["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare"]].copy() 

knn_imputer = KNNImputer(n_neighbors=3)
filled_df = pd.DataFrame(data=knn_imputer.fit_transform(X),
                         columns=["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare"])
filled_df.loc[missing_age]

,Survived,Pclass,Age,SibSp,Parch,Fare
5,0.0,3.0,23.666667,0.0,0.0,8.4583
17,1.0,2.0,32.833333,0.0,0.0,13.0000
19,1.0,3.0,16.666667,0.0,0.0,7.2250
26,0.0,3.0,38.500000,0.0,0.0,7.2250
28,1.0,3.0,24.666667,0.0,0.0,7.8792
...,...,...,...,...,...,...
859,0.0,3.0,24.666667,0.0,0.0,7.2292
863,0.0,3.0,38.333333,8.0,2.0,69.5500
868,0.0,3.0,24.000000,0.0,0.0,9.5000
878,0.0,3.0,31.333333,0.0,0.0,7.8958


The [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) for `KNNImputer` states "Each sample’s missing values are imputed using the mean value from n_neighbors nearest neighbors found in the training set. Two samples are close if the features that neither is missing are close.". Hence, I removed all the object datatypes because this will raise an error in the for this algorithm. 